In [41]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook
import time
import datetime
pd.options.mode.chained_assignment = None  # default='warn'


In [42]:
def change_cat_to_reeest(cell):
    """
    Функция для замены значений категорий используемыъ в таблице 3 на значения используемые в реестре УПП
    """
    temp_str = cell.strip() # очищаем от пробелов на всякий случай
    if temp_str == '120':
        return '2'
    elif temp_str == '100':
        return '1'
    elif temp_str in ('130','131','132','133','134','135','136','','','','','','','','','','','','','',):
        return '3'
    elif temp_str in ('141','142','143','144','145','146','147','148','149','150','151','152'):
        return '4'
    else:
        return '0'
    
    

In [43]:
def create_doc_convert_date(cell):
    """
    Функция для конвертации даты при создании документов
    :param cell:
    :return:
    """
    try:
        string_date = datetime.datetime.strftime(cell, '%d.%m.%Y')
        return string_date
    except ValueError:
        return ''

In [44]:
file_transfer_to_upp = 'data/Перенос из таблицы 3 в реестр УПП/Таблица 3 с дополнительными колонками..xlsx'
file_transfer_reestr = 'data/Перенос из таблицы 3 в реестр УПП/2022-10-27_64_Реестр УПП с дополнительными колонками.xlsx'
path_to_end_folder = 'data/'
header_reestr = 'data/Перенос из таблицы 3 в реестр УПП/Заголовок Реестр УПП от 25.10.xlsx'

In [45]:
# Загружаем датафреймы
df_upp = pd.read_excel(file_transfer_reestr,skiprows=8)
df_table_3 = pd.read_excel(file_transfer_to_upp,skiprows=6,usecols=[4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28,
                                                           29,32])


In [46]:
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df_upp.columns = list(map(str,list(df_upp.columns)))
df_table_3.columns = list(map(str,list(df_table_3.columns)))

# Очищаем на всякий случай от пробельных символов
df_upp.columns = list(map(lambda x:x.strip(),list(df_upp.columns)))
df_table_3.columns = list(map(lambda x:x.strip(),list(df_table_3.columns)))

In [49]:
# Фильтруем датафрейм отбирая только те записи в которых есть 1 в графе 30
transfer_df=df_table_3[df_table_3['30'] == 1]


# перемещаем площадь выдела
transfer_df.insert(7,'12',transfer_df['33'])

# удаляем лишний столбец с площадью выдела и признаков внесения в реестр
transfer_df.drop(columns=['30','33'],inplace=True)

In [50]:
transfer_df.columns

Index(['5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '22', '23', '24', '25', '26', '27', '28', '29'],
      dtype='object')

In [51]:
transfer_df['17'] =transfer_df['17'].astype(str) # Приводим колонку к строковому формату

In [52]:
transfer_df['17'] =transfer_df['17'].apply(change_cat_to_reeest) # Заменяем категории таблицы 3 на категории Реестра УПП

In [53]:
transfer_df.columns

Index(['5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '22', '23', '24', '25', '26', '27', '28', '29'],
      dtype='object')

In [54]:
transfer_df.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']

In [55]:
transfer_df.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24'],
      dtype='object')

In [56]:
itog_df = pd.concat([df_upp,transfer_df],ignore_index=True)

In [58]:
itog_df.sort_values(['1','2','3','4','5',
                     ],inplace=True)

In [59]:
# Приводим даты к нормальному виду ДД.ММ.ГГГГ
itog_df['17'] = pd.to_datetime(itog_df['17'],errors='coerce',dayfirst=True)
itog_df['22'] = pd.to_datetime(itog_df['22'],errors='coerce',dayfirst=True)
itog_df['17'] = itog_df['17'].apply(create_doc_convert_date)
itog_df['22'] = itog_df['22'].apply(create_doc_convert_date)


In [62]:
# Получаем текущую дату
current_time = time.strftime('%H_%M_%S %d.%m.%Y')
# Сохраняем отчет
# Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
wb = load_workbook(header_reestr) # Создаем объект
# Записываем результаты
for row in dataframe_to_rows(itog_df,index=False,header=False):
    wb['Реестр УПП'].append(row)

In [63]:
wb.save(f'{path_to_end_folder}/Реестр УПП с добавлением данных из таблицы 3 {current_time}.xlsx')